---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [101]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [102]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [103]:
import pandas as pd
import numpy as np
import re

def get_list_of_university_towns():
    UnTown = pd.read_csv('university_towns.txt', sep = '\n', header=None, squeeze = True, skipinitialspace=True)
    rename = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    rename = {val:key for (key,val) in rename.items()}
    town_in_state = {}

    for line in UnTown:
        if '[edit]'in line:
            state_name = line.replace('[edit]','')
        else:
            town_in_state[line] = state_name

    university_towns = (pd.Series(town_in_state, name = 'State')
                        .reset_index()
                        .set_index('State')
                        .reset_index()
                        .rename(columns={'index':'RegionName'}))

    #university_towns['RegionName'] = university_towns['RegionName'].apply(lambda x: re.sub(r'\d+|\([^()]*\)|\[[^()]*\]', '', x).strip(' '))
    university_towns['RegionName'] = university_towns['RegionName'].apply(lambda x: x.split(' (')[0].strip())

    return university_towns.sort_values('State')

get_list_of_university_towns()


,State,RegionName
176,Alabama,Florence
30,Alabama,Auburn
275,Alabama,Livingston
315,Alabama,Montevallo
240,Alabama,Jacksonville
470,Alabama,Tuscaloosa
466,Alabama,Troy
471,Alabama,Tuskegee
160,Alaska,Fairbanks
173,Arizona,Flagstaff


In [104]:
def get_recession_start():
    GPD = pd.read_excel('gdplev.xls', header=[4])
    GPD = (GPD.drop([0,1])
              .drop(['Unnamed: 0','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Unnamed: 3','GDP in billions of current dollars.1', 'Unnamed: 7'], axis=1)
              .rename(columns={'Unnamed: 4': 'Year_quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'}))

    GPD = (GPD.where(GPD['Year_quarter']>='2000q1')
              .dropna())

    GPD['for delta'] = GPD['GDP in billions of chained 2009 dollars'].shift(1)
    GPD['Delta'] = np.sign(GPD['GDP in billions of chained 2009 dollars'] - GPD['for delta'])

    res_start = GPD.where((GPD['Delta'] == GPD['Delta'].shift(-1)) & (GPD['Delta'] == -1)).dropna()['Year_quarter'].iloc[0]
    
    return res_start
get_recession_start()

#'''Returns the year and quarter of the recession start time as a 
 #   string value in a format such as 2005q3'''

'2008q3'

In [105]:
def get_recession_end():
    GPD = pd.read_excel('gdplev.xls', header=[4])
    GPD = (GPD.drop([0,1])
              .drop(['Unnamed: 0','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Unnamed: 3','GDP in billions of current dollars.1', 'Unnamed: 7'], axis=1)
              .rename(columns={'Unnamed: 4': 'Year_quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'}))
    GPD = (GPD.where(GPD['Year_quarter']>='2000q1')
              .dropna())

    GPD['for delta'] = GPD['GDP in billions of chained 2009 dollars'].shift(1)
    GPD['Delta'] = np.sign(GPD['GDP in billions of chained 2009 dollars'] - GPD['for delta'])

    res_start = GPD.where((GPD['Delta'] == GPD['Delta'].shift(-1)) & (GPD['Delta'] == -1)).dropna()['Year_quarter'].iloc[0]
    res_end = GPD.where((GPD['Delta'] == GPD['Delta'].shift(-1)) & (GPD['Delta'] == 1) & (GPD['Year_quarter']>res_start)).dropna()['Year_quarter'].iloc[1]
           
    return res_end

get_recession_end()

#'''Returns the year and quarter of the recession end time as a 
 #   string value in a format such as 2005q3'''

'2009q4'

In [106]:
def get_recession_bottom():
    GPD = pd.read_excel('gdplev.xls', header=[4])
    GPD = (GPD.drop([0,1])
              .drop(['Unnamed: 0','GDP in billions of current dollars','GDP in billions of chained 2009 dollars','Unnamed: 3','GDP in billions of current dollars.1', 'Unnamed: 7'], axis=1)
              .rename(columns={'Unnamed: 4': 'Year_quarter', 'GDP in billions of chained 2009 dollars.1': 'GDP in billions of chained 2009 dollars'}))
    GPD = (GPD.where(GPD['Year_quarter']>='2000q1')
              .dropna())

    GPD['for delta'] = GPD['GDP in billions of chained 2009 dollars'].shift(1)
    GPD['Delta'] = np.sign(GPD['GDP in billions of chained 2009 dollars'] - GPD['for delta'])

    res_start = GPD.where((GPD['Delta'] == GPD['Delta'].shift(-1)) & (GPD['Delta'] == -1)).dropna()['Year_quarter'].iloc[0]
    res_end = GPD.where((GPD['Delta'] == GPD['Delta'].shift(-1)) & (GPD['Delta'] == 1) & (GPD['Year_quarter']>res_start)).dropna()['Year_quarter'].iloc[0]
    
    min_GPD = np.min(GPD[(GPD['Year_quarter']>=res_start) & (GPD['Year_quarter']<=res_end)]['GDP in billions of chained 2009 dollars'])
    rec_bottom = GPD[GPD['GDP in billions of chained 2009 dollars'] == min_GPD]['Year_quarter'].iloc[0]
    
    return rec_bottom

get_recession_bottom()

#'''Returns the year and quarter of the recession bottom time as a 
 #   string value in a format such as 2005q3'''

'2009q2'

In [107]:
def convert_housing_data_to_quarters():
    usecols=[i for i in range(51, 251)]
    usecols.extend([1,2])
    House = pd.read_csv('City_Zhvi_AllHomes.csv', usecols=usecols)
    House = House.sort_values('State')
    rename = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    House = (House.set_index(['State', 'RegionName'])
                 .rename(rename))
    House.columns = pd.to_datetime(House.columns)
    House = (House.resample('Q', axis=1).mean()
                  .rename(columns = lambda col: '{}q{}'.format(col.year, col.quarter)))
    
    return House

convert_housing_data_to_quarters()





#'''Converts the housing data to quarters and returns it as mean 
 #   values in a dataframe. This dataframe should be a dataframe with
  #  columns for 2000q1 through 2016q3, and should have a multi-index
#    in the shape of ["State","RegionName"].
    
#    Note: Quarters are defined in the assignment description, they are
#    not arbitrary three month periods.
    
 #   The resulting dataframe should have 67 columns, and 10,730 rows.
  #  '''


2000q1         2000q2         2000q3  \
State         RegionName                                                      
Alaska        Kenai                       NaN            NaN            NaN   
              Ketchikan                   NaN            NaN            NaN   
              Anchor Point                NaN            NaN            NaN   
              Anchorage         174633.333333  175266.666667  179566.666667   
              Palmer            168766.666667  170566.666667  174233.333333   
              Seward                      NaN            NaN            NaN   
              Kodiak                      NaN            NaN            NaN   
              Tanaina           160500.000000  163500.000000  166366.666667   
              Lakes             172733.333333  174133.333333  177266.666667   
              North Pole        157466.666667  159100.000000  162133.333333   
              Homer                       NaN            NaN            NaN   
              Soldotna                    NaN            NaN            NaN   
              Juneau            192466.666667  194300.000000  195166.666667   
              Fairbanks         163200.000000  165033.333333  169300.000000   
Alabama       Birmingham         54033.333333   54400.000000   54966.666667   
              Gurley                      NaN            NaN            NaN   
              Fosters           107500.000000  110200.000000  111400.000000   
              Hope Hull         111666.666667  111800.000000  112966.666667   
              Lipscomb                    NaN            NaN            NaN   
              Guntersville       92933.333333   93500.000000   93600.000000   
              Meadowbrook       200500.000000  203000.000000  206800.000000   
              Northport         125866.666667  126366.666667  126633.333333   
              Morris            124066.666667  124500.000000  124233.333333   
              Good Hope                   NaN            NaN            NaN   
              Orange Beach      221633.333333  222133.333333  224033.333333   
              Robertsdale        86833.333333   87666.666667   89000.000000   
              Cottondale        100833.333333  102633.333333  104766.666667   
              Titus                       NaN            NaN            NaN   
              Leeds              93833.333333   94433.333333   95066.666667   
              Homewood          158333.333333  159666.666667  161100.000000   
...                                       ...            ...            ...   
Wisconsin     Erin              208566.666667  209100.000000  209666.666667   
West Virginia Nitro                       NaN            NaN            NaN   
              Eskdale                     NaN            NaN            NaN   
              Mineral Wells      75533.333333   77233.333333   81966.666667   
              Cross Lanes        78133.333333   78066.666667   79233.333333   
              Dunbar                      NaN            NaN            NaN   
              Vienna             72266.666667   74533.333333   75333.333333   
              Saint Albans       63833.333333   64700.000000   65200.000000   
              South Charleston            NaN            NaN            NaN   
              Parkersburg        54900.000000   55833.333333   56633.333333   
              Chesapeake                  NaN            NaN            NaN   
              East Bank                   NaN            NaN            NaN   
              Belle              51066.666667   51300.000000   52200.000000   
              Marmet                      NaN            NaN            NaN   
              Upper Falls        53033.333333   52333.333333   52100.000000   
              Waverly            62366.666667   62933.333333   65300.000000   
              Sissonville        57200.000000   58200.000000   61200.000000   
              Charleston         59533.333333   59366.666667   59366.666667   
          

In [109]:
def run_ttest():
    res_before_start = '2008q2'
    res_bottom = get_recession_bottom()
    House_price = pd.DataFrame(convert_housing_data_to_quarters()[res_bottom]/convert_housing_data_to_quarters()[res_before_start])
    UT_data = get_list_of_university_towns()

    UT_data = UT_data.set_index(['State', 'RegionName'])
    
    UT_House_price = House_price.merge(UT_data, how='inner', left_index = True, right_index = True)
    NUT_House_price = House_price.drop(UT_House_price.index)

    Ttest_Result = ttest_ind(UT_House_price, NUT_House_price, nan_policy='omit')
    p = float(Ttest_Result[1].data)
    different = True if p<0.01 else False
    better = 'university town' if (float(UT_House_price.mean()) > float(NUT_House_price.mean())) else 'non-university town'
        
    return (different, p, better)


run_ttest()

(True, 0.0033629228768433245, 'university town')